In [216]:
# Import modules needed
import pandas as pd
import os
import sqlite3
from sqlite3 import Error
# import requests
# from config import rapidapi_host, rapidapi_key
# import pprint

# # Identify database path
# db_path = os.path.abspath('./../static/data/sqlite.db')
# db_path

# Web scraping modules
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
from bs4 import BeautifulSoup as soup

In [217]:
# Read in Florida Zip Code list
# This list was manually copied from https://www.zipcodestogo.com/Florida/ into a csv
file_path = os.path.abspath('./../static/data/FloridaZipCodes.csv')
zipcode_ls = pd.read_csv(file_path)['Zip Code'].unique().tolist()
# zipcode_ls

In [215]:
# Loop through zip code list & scrape rent data
base_url = 'https://www.rentdata.org/lookup?zip='

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# zipcode_ls = [32004]

zipcode_df = pd.DataFrame()
for zipcode in zipcode_ls:
    
    try: 
        browser.visit(f'{base_url}{zipcode}')
        # Convert the browser html to a soup object
        html = browser.html
        soup = soup(html, 'html.parser')

        # Find the div with the table
        calc_results = soup.select_one('div.calc-results')

        # Find metro & year
        metro = calc_results.find('h4',class_='calc-title').text

        # Find table
        table_html = soup.select_one('table.text-center')
        table_rows = table_html.find('tbody').find_all('tr')
        for row in table_rows:
            # Reset variables
            rent_0b = ''
            rent_1b = ''
            rent_2b = ''
            rent_3b = ''
            rent_4b = ''

            if row == '<tr></tr>':
                pass
            else:
                th = row.find('th',class_='show-for-small-only')
                td = row.find('td')
                if th == None:
                    th = ''
                elif th.text == '0-BR':
                    rent_0b = td.text
                elif th.text == '1-BR':
                    rent_1b = td.text
                elif th.text == '2-BR':
                    rent_2b = td.text
                elif th.text == '3-BR':
                    rent_3b = td.text
                elif th.text == '4-BR':
                    rent_4b = td.text

        df = pd.DataFrame({'zipcode' : [zipcode]
                        ,'metro' : [metro]
                        ,'rent_0b' : [rent_0b]
                        ,'rent_1b' : [rent_1b]
                        ,'rent_2b' : [rent_2b]
                        ,'rent_3b' : [rent_3b]
                        ,'rent_4b' : [rent_4b]})
        zipcode_df = zipcode_df.append(df)

        browser.quit()
    except:
        print(f'{zipcode} failed')

SyntaxError: EOL while scanning string literal (<ipython-input-215-c862789586f1>, line 59)

In [22]:
zipcode_df

'c:\\Users\\chris\\dev\\cgangi2216\\Housing_Valuation\\Data_Cleansing\\static\\data\\sqlite.db'

In [ ]:
zipcode_df.to_csv('cleaned_csv', index=False)